In [2]:
from expt_settings.configs import ExperimentConfig

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import datetime
import gc
import glob
import os
import shutil
import sys

import numpy as np
import pandas as pd
import pyunpack
import wget

In [3]:
os.getcwd()

NameError: name 'os' is not defined

# Unzip

In [5]:
import pyunpack

In [6]:
def unzip(zip_path, output_file, data_folder):
  """Unzips files and checks successful completion."""

  print('Unzipping file: {}'.format(zip_path))
  pyunpack.Archive(zip_path).extractall(data_folder)

  # Checks if unzip was successful
  if not os.path.exists(output_file):
    raise ValueError(
        'Error in unzipping process! {} not found.'.format(output_file))

In [7]:
url = 'https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data'

In [8]:
data_folder ='0615_dataset'

In [9]:
# Save manual download to root folder to avoid deleting when re-processing.
zip_file = os.path.join(data_folder,
                          'favorita-grocery-sales-forecasting.zip')

In [10]:
zip_file

'0615_dataset\\favorita-grocery-sales-forecasting.zip'

In [8]:
if not os.path.exists(zip_file):
    raise ValueError(
        'Favorita zip file not found in {}!'.format(zip_file) +
        ' Please manually download data from Kaggle @ {}'.format(url))

In [15]:
# Unpack main zip file
outputs_file = os.path.join(data_folder, 'train.csv.7z')
unzip(zip_file, outputs_file, data_folder)

Unzipping file: 0615_dataset\favorita-grocery-sales-forecasting.zip


In [32]:
# Unpack individually zipped files
for file in glob.glob(os.path.join(data_folder, '*.7z')):
    csv_file = file.replace('.7z', '')
    unzip(file, csv_file, data_folder)
    print('Unzipping complete, commencing data processing...')

Unzipping file: 0615_dataset\holidays_events.csv.7z


ValueError: patool not found! Please install patool!

In [16]:
print('Unzipping complete, commencing data processing...')

Unzipping complete, commencing data processing...


# Sampling

In [57]:
# Extract only a subset of data to save/process for efficiency
start_date = datetime.datetime(2013, 1, 1)
end_date = datetime.datetime(2015, 4, 1)

print('Regenerating data...')

Regenerating data...


In [63]:
# load temporal data
tettmporal = pd.read_csv(os.path.join(data_folder, 'test.csv'), index_col=0)# nrows=100000

In [70]:
tettmporal.iloc[10000]

date           2017-08-16
store_nbr               3
item_nbr          1400334
onpromotion         False
traj_id         3_1400334
Name: 125507040, dtype: object

In [66]:
tettmporal['traj_id'] = tettmporal['store_nbr'].apply(
      str) + '_' + tettmporal['item_nbr'].apply(str)
tettmporal['traj_id'].tail(50)

id
128867454    54_2123790
128867455    54_2123791
128867456    54_2123839
128867457    54_2123859
128867458    54_2123863
128867459    54_2124052
128867460    54_2124705
128867461    54_2125814
128867462    54_2126696
128867463    54_2126842
128867464    54_2126944
128867465    54_2127024
128867466    54_2127069
128867467    54_2127114
128867468    54_2127175
128867469    54_2127206
128867470    54_2127544
128867471    54_2127651
128867472    54_2127733
128867473    54_2127825
128867474    54_2127921
128867475    54_2127992
128867476    54_2128628
128867477    54_2128799
128867478    54_2129334
128867479    54_2129350
128867480    54_2129387
128867481    54_2129515
128867482    54_2129616
128867483    54_2129678
128867484    54_2129786
128867485    54_2129790
128867486    54_2129892
128867487    54_2129994
128867488    54_2130131
128867489    54_2130219
128867490    54_2130265
128867491    54_2130352
128867492    54_2130474
128867493    54_2130521
128867494    54_2130526
128867495    

In [27]:
temporal['traj_id'] = temporal['store_nbr'].apply(
      str) + '_' + temporal['item_nbr'].apply(str)
traj_id_list = temporal['traj_id'].head(10).to_list()

['25_103665',
 '25_105574',
 '25_105575',
 '25_108079',
 '25_108701',
 '25_108786',
 '25_108797',
 '25_108952',
 '25_111397',
 '25_114790']

In [28]:
store_info = pd.read_csv(os.path.join(data_folder, 'stores.csv'), index_col=0)
oil = pd.read_csv(
      os.path.join(data_folder, 'oil.csv'), index_col=0).iloc[:, 0]
holidays = pd.read_csv(os.path.join(data_folder, 'holidays_events.csv'))
items = pd.read_csv(os.path.join(data_folder, 'items.csv'), index_col=0)
transactions = pd.read_csv(os.path.join(data_folder, 'transactions.csv'))

Pick ['traj_id'] == ['25_103665', '1_103520', '1_103665', '1_96995', '3_1400334']

In [72]:
chunksize = 10**5
#temporal = pd.DataFrame()
for chunk in pd.read_csv(os.path.join(data_folder, 'train.csv'), index_col=0, chunksize=chunksize):
    chunk['date'] = pd.to_datetime(chunk['date'])

    chunk['traj_id'] = chunk['store_nbr'].apply(
      str) + '_' + chunk['item_nbr'].apply(str)
    chunk = chunk[chunk['traj_id']=='3_1400334']
    #chunk_temp = chunk.groupby(['traj_id']).sample(100).reset_index()
    temporal = temporal.append(chunk,ignore_index=True)

C:\Users\pan\AppData\Local\Temp\ipykernel_26356\282422833.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temporal = temporal.append(chunk,ignore_index=True)
C:\Users\pan\AppData\Local\Temp\ipykernel_26356\282422833.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temporal = temporal.append(chunk,ignore_index=True)
C:\Users\pan\AppData\Local\Temp\ipykernel_26356\282422833.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temporal = temporal.append(chunk,ignore_index=True)
C:\Users\pan\AppData\Local\Temp\ipykernel_26356\282422833.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temporal = temporal.append(chunk,ignore_inde

KeyboardInterrupt: 

In [76]:
temporal['traj_id'].unique()

array(['25_103665', '1_103520', '1_103665', '1_96995', '3_1400334'],
      dtype=object)

In [77]:
temporal.to_csv('favorita_original.csv',index=False)

In [78]:
  # Filter dates to reduce storage space requirements
if start_date is not None:
    temporal = temporal[(temporal['date'] >= start_date)]
if end_date is not None:
    temporal = temporal[(temporal['date'] < end_date)]

In [80]:
# Add trajectory identifier
temporal['traj_id'] = temporal['store_nbr'].apply(
      str) + '_' + temporal['item_nbr'].apply(str)
temporal['unique_id'] = temporal['traj_id'] + '_' + temporal['date'].apply(
      str)

In [81]:
# Remove all IDs with negative returns
print('Removing returns data')
min_returns = temporal['unit_sales'].groupby(temporal['traj_id']).min()
valid_ids = set(min_returns[min_returns >= 0].index)
selector = temporal['traj_id'].apply(lambda traj_id: traj_id in valid_ids)
new_temporal = temporal[selector].copy()
del temporal
gc.collect()
temporal = new_temporal
temporal['open'] = 1

Removing returns data


# Resampling

In [82]:
# Resampling
print('Resampling to regular grid')
resampled_dfs = []
for traj_id, raw_sub_df in temporal.groupby('traj_id'):
    print('Resampling', traj_id)
    sub_df = raw_sub_df.set_index('date', drop=True).copy()
    sub_df = sub_df.resample('1d').last()
    sub_df['date'] = sub_df.index
    sub_df[['store_nbr', 'item_nbr', 'onpromotion']] \
        = sub_df[['store_nbr', 'item_nbr', 'onpromotion']].fillna(method='ffill')
    sub_df['open'] = sub_df['open'].fillna(
        0)  # flag where sales data is unknown
    sub_df['log_sales'] = np.log(sub_df['unit_sales'])

    resampled_dfs.append(sub_df.reset_index(drop=True))

Resampling to regular grid
Resampling 1_103520
Resampling 1_103665
Resampling 1_96995
Resampling 25_103665
Resampling 3_1400334


# Add new variables

In [ ]:
new_temporal = pd.concat(resampled_dfs, axis=0)
del temporal
gc.collect()
temporal = new_temporal

In [84]:
# Oil (based on 'date)
print('Adding oil')
oil.name = 'oil'
oil.index = pd.to_datetime(oil.index)
temporal = temporal.join( 
      oil.fillna(method='ffill'), on='date', how='left')  # oil.loc[dates].fillna(method='ffill')
temporal['oil'] = temporal['oil'].fillna(-1)

Adding oil


In [85]:
# store (based on 'store_nbr')
print('Adding store info')
temporal = temporal.join(store_info, on='store_nbr', how='left')

Adding store info


In [86]:
print('Adding item info')
temporal = temporal.join(items, on='item_nbr', how='left')

transactions['date'] = pd.to_datetime(transactions['date'])
temporal = temporal.merge(
      transactions,
      left_on=['date', 'store_nbr'],
      right_on=['date', 'store_nbr'],
      how='left')
temporal['transactions'] = temporal['transactions'].fillna(-1)

Adding item info


In [87]:
# Additional date info
temporal['day_of_week'] = pd.to_datetime(temporal['date'].values).dayofweek
temporal['day_of_month'] = pd.to_datetime(temporal['date'].values).day
temporal['month'] = pd.to_datetime(temporal['date'].values).month

In [88]:
# Add holiday info
print('Adding holidays')
holiday_subset = holidays[holidays['transferred'].apply(
      lambda x: not x)].copy()
holiday_subset.columns = [
      s if s != 'type' else 'holiday_type' for s in holiday_subset.columns
  ]
holiday_subset['date'] = pd.to_datetime(holiday_subset['date'])
local_holidays = holiday_subset[holiday_subset['locale'] == 'Local']
regional_holidays = holiday_subset[holiday_subset['locale'] == 'Regional']
national_holidays = holiday_subset[holiday_subset['locale'] == 'National']

temporal['national_hol'] = temporal.merge(
      national_holidays, left_on=['date'], right_on=['date'],
      how='left')['description'].fillna('')
temporal['regional_hol'] = temporal.merge(
      regional_holidays,
      left_on=['state', 'date'],
      right_on=['locale_name', 'date'],
      how='left')['description'].fillna('')
temporal['local_hol'] = temporal.merge(
      local_holidays,
      left_on=['city', 'date'],
      right_on=['locale_name', 'date'],
      how='left')['description'].fillna('')

temporal.sort_values('unique_id', inplace=True)

Adding holidays


In [97]:
temporal[temporal['traj_id']=='1_103520']
# '25_103665', '1_103520', '1_103665', '1_96995', '3_1400334'

,store_nbr,item_nbr,unit_sales,onpromotion,traj_id,unique_id,open,date,log_sales,oil,...,family,class,perishable,transactions,day_of_week,day_of_month,month,national_hol,regional_hol,local_hol
0,1.0,103520.0,2.0,None,1_103520,1_103520_2013-01-04 00:00:00,1.0,2013-01-04,0.693147,93.12,...,GROCERY I,1028,0,1863.0,4,4,1,,,
1,1.0,103520.0,3.0,None,1_103520,1_103520_2013-01-05 00:00:00,1.0,2013-01-05,1.098612,-1.00,...,GROCERY I,1028,0,1509.0,5,5,1,Recupero puente Navidad,,
3,1.0,103520.0,2.0,None,1_103520,1_103520_2013-01-07 00:00:00,1.0,2013-01-07,0.693147,93.20,...,GROCERY I,1028,0,1807.0,0,7,1,,,
4,1.0,103520.0,6.0,None,1_103520,1_103520_2013-01-08 00:00:00,1.0,2013-01-08,1.791759,93.21,...,GROCERY I,1028,0,1869.0,1,8,1,,,
5,1.0,103520.0,3.0,None,1_103520,1_103520_2013-01-09 00:00:00,1.0,2013-01-09,1.098612,93.08,...,GROCERY I,1028,0,1910.0,2,9,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,1.0,103520.0,2.0,False,1_103520,1_103520_2015-03-25 00:00:00,1.0,2015-03-25,0.693147,48.75,...,GROCERY I,1028,0,1890.0,2,25,3,,,
811,1.0,103520.0,2.0,False,1_103520,1_103520_2015-03-26 00:00:00,1.0,2015-03-26,0.693147,51.41,...,GROCERY I,1028,0,1607.0,3,26,3,,,
813,1.0,103520.0,2.0,False,1_103520,1_103520_2015-03-28 00:00:00,1.0,2015-03-28,0.693147,-1.00,...,GROCERY I,1028,0,1279.0,5,28,3,,,
815,1.0,103520.0,1.0,False,1_103520,1_103520_2015-03-30 00:00:00,1.0,2015-03-30,0.000000,48.66,...,GROCERY I,1028,0,1753.0,0,30,3,,,


In [91]:
temporal['unit_sales'].isna().sum()

1007

# Extract file

In [98]:
data_csv_path = 'favorita_out.csv'
print('Saving processed file to {}'.format(data_csv_path))
temporal.to_csv(data_csv_path)

Saving processed file to favorita_out.csv


In [100]:
experiment_names = 'favorita'
parser = argparse.ArgumentParser(description='Data download configs')
parser.add_argument(
        'expt_name',
        metavar='e',
        type=str,
        nargs='?',
        choices=experiment_names,
        help='Experiment Name. Default={}'.format(','.join(experiment_names)))
parser.add_argument(
        'output_folder',
        metavar='f',
        type=str,
        nargs='?',
        default='.',
        help='Path to folder for data download')
parser.add_argument(
        'force_download',
        metavar='r',
        type=str,
        nargs='?',
        choices=['yes', 'no'],
        default='no',
        help='Whether to re-run data download')
args = parser.parse_known_args()[0]

In [102]:
root_folder = None if args.output_folder == '.' else args.output_folder

In [104]:
args

Namespace(expt_name=None, output_folder='.', force_download='no')